# SOURCE CODE
# https://gist.github.com/victorkohler/f48ea6512058719ba52053851fedc745

In [1]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

# Data Preprocessing for Matrix User x Item

In [2]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

In [3]:
#np.shape(train_data)
np.shape(test_data)

(3782335, 12)

In [4]:
train = train_data.append(test_data)
np.shape(train)

MemoryError: 

In [ ]:
item_metadata = pd.read_csv('item_metadata.csv', sep=',', engine='python')

In [ ]:
train_v2 = train[train['action_type']=='clickout item']
train_v2= train_v2[train_v2.reference.notnull()]
train_v2.head()

In [ ]:
train_v2.dtypes

In [ ]:
train_v2["reference"]= train_v2["reference"].astype(int)
train_v2["step"]= train_v2["step"].astype(int)
train_v2.dtypes

In [ ]:
train_v2.head()
#np.shape(train_v2)

In [ ]:
#train_v3 = train_v2.head(10000)
train_v3 = train_v2
train_v3.head()
np.shape(train_v3)

In [ ]:
data = train_v3[['user_id','reference']]
data.head()
#np.shape(data)

In [ ]:
data=data.groupby(['user_id','reference']).size().reset_index()
data.columns = ['user', 'item', 'click_count']
data.head()

In [ ]:
user_id_table = data['user']
user_id_table = pd.DataFrame(user_id_table)
user_id_table = pd.DataFrame(user_id_table['user'].unique())
user_id_table.columns = ['user']
user_id_table['user_id'] = dict(enumerate(user_id_table['user'], start=0))
user_id_table.head()
#np.shape(user_id_table)

In [ ]:
item_id_table = data['item']
item_id_table = pd.DataFrame(item_id_table)
item_id_table = pd.DataFrame(item_id_table['item'].unique())
item_id_table.columns = ['item']
item_id_table['item_id'] = dict(enumerate(item_id_table['item'], start=0))
item_id_table.head()
#np.shape(item_id_table)

In [ ]:
data = data.merge(user_id_table, how='left', left_on='user', right_on = 'user').merge(item_id_table, how='left', left_on='item', right_on = 'item')
data.head()

In [ ]:
import scipy.sparse as sparse
import numpy as np
import random

def mini_batch_generator(graph,batch_size,row_length,col_length):
    i = random.randint(0, row_length-batch_size)
    j = random.randint(0,col_length-batch_size)
    return graph[i:i+batch_size,j:j+batch_size]

def negative_sampling(E,row_length,col_length,batch_size):
    S = sparse.dok_matrix((row_length+1,col_length+1)) #create empty negtive feedack graph
    for i in range(batch_size):
        a = random.randint(0, row_length)
        b = random.randint(0,col_length)
        randomEdge = (a,b) #randomly sample edges from the implicit feedback graph
        if (a,b) not in E:
            S[a,b]=1
    return S.tocoo()

def compute_e_ife(theta_u,theta_v,G_items,S_items):
    sumG = sumS = 0
    for (i,j) in G_items:
        currVal = np.dot(theta_u[i],np.transpose(theta_v[j]))
        h = 1/(1+np.exp(-currVal))
        sumG += np.log(h)
    for (i,j) in S_items:
        currVal = np.dot(theta_u[i],np.transpose(theta_v[j]))
        h = 1/(1+np.exp(-currVal))
        sumS += np.log(1-h)
    return sumG + sumS
            
def update_embeddings(theta_u,theta_v,G_items,S_items,a):
    sumGu = sumSu = sumGv = sumSv = 0
    for (i,j) in G_items:
        currVal = np.dot(theta_u[i],np.transpose(theta_v[j]))
        currVal = 1/(1+np.exp(-currVal))
        currVal = (1 - currVal)
        uVal = np.dot(currVal,theta_v[j])  
        vVal = np.dot(currVal,theta_u[i])
        sumGu += uVal
        sumGv += vVal
    for (i,j) in S_items:
        currVal = np.dot(theta_u[i], np.transpose(theta_v[j]))
        currVal = 1/(1+np.exp(-currVal))
        currVal = (-currVal)
        uVal = np.dot(currVal,theta_v[j])  
        vVal = np.dot(currVal,theta_u[i])
        sumSu += uVal
        sumSv += vVal
    theta_u = theta_u + (a*sumGu + a*sumSu) #update user embeddings
    theta_v = theta_v + (a*sumGv + a*sumSv) #update item embeddings
    
def implicit_feedback_embedding(G,S,b,a,k,row_length,col_length):
    #initiliase list of vector embeddings randomly
    theta_u = list()
    theta_v = list()
    for i in range(row_length+1):
        theta_u.append(np.random.rand(k))
    for i in range(col_length+1):
        theta_v.append(np.random.rand(k))   
    tolerance = 2 #convergence parameter epsilon
    convergence_difference = 5
    G_items = set(zip(G.row,G.col)) #get iterator for row,col pair with non-zero values
    S = sparse.dok_matrix((row_length+1,col_length+1)) #create empty negtive feedack graph
    #e_ife = compute_e_ife(theta_u,theta_v,G,S) #initialise e_ife
    G = G.tocsr()
    
    while abs(convergence_difference) > tolerance: #while embedding has not converged
        G_mini = mini_batch_generator(G,batch_size,row_length,col_length)
        S = negative_sampling(G_items,row_length,col_length,batch_size) #make negative feedback graph
        #print(G_mini.count_nonzero())
        #print(S.count_nonzero())
        G_mini = G.tocoo()
        #S.tocoo()
        S_items = set(zip(S.row,S.col))
        G_mini_items = set(zip(G_mini.row,G_mini.col))
        update_embeddings(theta_u,theta_v,G_mini_items,S_items,a)
        #new_ife = compute_e_ife(theta_u,theta_v,G,S)
        convergence_difference -= 2
     
#set parameters
batch_size = 20
learning_rate = 1
dimensionality = 5
#adjacency matrix representing the implicit Feedback Graph G
G = sparse.coo_matrix((data['click_count'].astype(float), (data['user_id'], data['item_id'])))
row_length = max(G.row)
col_length = max(G.col)
print("embedding...")
implicit_feedback_embedding(G,S,batch_size,learning_rate,dimensionality,row_length,col_length)
print("done")

G = G.tocsr()
G = G[:5000, :5000]
G = G.todense() #gives a numpy matrix